In [ ]:
# ✅ 1. 라이브러리 설치
!pip install --upgrade openai gspread gspread_dataframe oauth2client

# ✅ 2. 인증 및 모듈 임포트
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
from openai import OpenAI

creds, _ = default()
gc = gspread.authorize(creds)

# ✅ 3. 구글 시트 연결
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1kkt336f1G-XqfDuwCUOnqpKlxTcnwLQy-XS4SQv6lM0/edit"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.sheet1
data = worksheet.get_all_values()
headers, rows = data[0], data[1:]
df = pd.DataFrame(rows, columns=headers)

# ✅ 4. OpenAI 클라이언트 설정
client = OpenAI(api_key="api키 입력")  # ← 너의 GPT API 키 입력

# ✅ 5. 산업군 목록
industry_list = [
    "반도체 공정", "회로설계", "임베디드SW",
    "HW", "인공지능", "통신", "로보틱스", "자동차"
]

# ✅ 6. GPT로 산업군 2개 추천 함수 (수강 과목만 기반)
def recommend_industries(subjects):
    if not subjects.strip():
        return "정보 없음"

    prompt = f"""
아래는 한 참가자의 수강한 전공 과목 목록입니다.
- 수강 과목: {subjects}

다음 산업군 목록 중 이 참가자에게 가장 적합한 산업군 2개를 골라주세요.
산업군 목록: {", ".join(industry_list)}

답변 형식: 산업1, 산업2 (쉼표로 구분된 2개)
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"에러: {str(e)}"

# ✅ 7. 결과 생성 및 저장
df["추천 산업 분야 2개"] = df["수강 한 전공 과목 (복수 응답 가능)"].apply(recommend_industries)

# ✅ 8. 결과를 새 시트에 저장
try:
    result_ws = sh.worksheet("추천결과")
except:
    result_ws = sh.add_worksheet(title="추천결과", rows="100", cols="20")

result_ws.clear()
set_with_dataframe(result_ws, df)
print("✅ 결과가 '추천결과' 시트에 저장되었습니다.")


✅ 결과가 '추천결과' 시트에 저장되었습니다.
